In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
from os import listdir
from os.path import isfile, join

In [5]:
%pip install Unidecode

import unidecode
# Function to remove Hungarian letters
def remove_hungarian_letters(text):
    return unidecode.unidecode(text)

Note: you may need to restart the kernel to use updated packages.


In [6]:
def to_int(list):
    tmp = []
    for val in list:
        if isinstance(val, str):
            tmp.append(int(float(val.replace(",", "."))))
    return(tmp)

In [13]:
import os

input_folder = './pontszamok'
output_folder = './rendezett_pontszamok'

dfs = []
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # Read the CSV file, Remove empty rows, Drop empty columns
        filepath = os.path.join(input_folder, filename)
        df = pd.read_csv(filepath).dropna(axis='columns', how='all')
        df = df.dropna(axis='index', how='all')

        # Convert column names to lowercase and remove Hungarian letters
        df.columns = [remove_hungarian_letters(
            str(col)).lower() for col in df.columns]

        # Transpose the specific CSV files before any processing
        if filename == '2016.csv':
            df = pd.read_csv(filepath, header=None, index_col=0).T
            cols = df.columns.tolist()
            df['helyezes'] = list(range(1, len(df) + 1))
            df = pd.concat([df['helyezes'], df[cols[1]], df[cols[0]],
                           df[cols[2:-1]]], axis=1)

            df.columns.values[1] = 'pontszam'
            df.columns.values[2] = 'csapat'

        elif filename == '2019.csv':
            df = pd.read_csv(filepath, header=None, index_col=0).T
            df['helyezes'] = list(range(1, len(df) + 1))
            cols = df.columns.tolist()
            df = pd.concat([df['helyezes'], df[cols[-2]],
                           df[cols[0]], df[cols[1:-2]]], axis=1)
            df.columns.values[1] = 'pontszam'
            df.columns.values[2] = 'csapat'
        elif filename == '2010.csv':
            df = pd.read_csv(filepath).rename(
                columns={'csapat': 'pontszam', 'pontszam': 'csapat'})
            df['csapat'], df['pontszam'] = df['pontszam'], df['csapat']
            df['pontszam'] = to_int(df['pontszam'])
        elif filename == '2013.csv' or filename == '2018.csv':
            df['helyezes'] = list(range(1, len(df) + 1))
            df = df[['helyezes', 'pontszam', 'csapat']]
        elif filename == '2012.csv':
            df = df.rename(columns={
                           'ssz.': 'helyezes', 'oszpont': 'pontszam', 'csapatok/programok': 'csapat'})
            cols = df.columns.tolist()
            df = df[[cols[0]] + [cols[-1]] + cols[1:-1]]
            df['pontszam'] = df['pontszam'].astype(int)
            df['helyezes'] = df['helyezes'].astype(int)
        elif filename == '2022.csv' or filename == '2023.csv':
            df = pd.read_csv(filepath, header=None, index_col=0).T
            header_row = df.iloc[0]  # Get the first row (header)
            df = df[1:]  # Remove the first row from the DataFrame
            df = df.rename(columns={'Összesen': 'pontszam'})
            df = df.sort_values(by=['pontszam'], ascending=False)
            df['helyezes'] = list(range(1, len(df) + 1))
            cols = df.columns.tolist()
            df = pd.concat([df['helyezes'], df[cols[-2]],
                           df[cols[0]], df[cols[1:-2]]], axis=1)
            df.columns.values[2] = 'csapat'
        # Save the modified DataFrame to a new CSV file in the output folder
        output_filepath = os.path.join(output_folder, filename)
        df.to_csv(output_filepath, index=False)

In [10]:
import pandas as pd
import unidecode

# reference csv
reference_csv = './rendezett_pontszamok/2011.csv'
reference_df = pd.read_csv(reference_csv)

# Process reference column names
reference_columns = [remove_hungarian_letters(
    col).lower() for col in reference_df.columns]

# Create the folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Read and process each CSV file
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        # Read the CSV file, Remove empty rows, Drop empty columns
        filepath = os.path.join(output_folder, filename)
        df = pd.read_csv(filepath)

     

        # Add missing columns with default values (NaN)
        for col in reference_columns:
            if col not in df.columns:
                df[col] = np.nan
        # Reorder the columns to match the reference columns
        df = df[reference_columns]

        # Save the modified DataFrame to a new CSV file in the output folder
        output_filepath = os.path.join(output_folder, filename)
        df.to_csv(output_filepath, index=False)

## Bemutato

Az adatok tisztitasa es egyseges formatumra valo hozasa fontos lepes annak erdekeben hogy elemezheto es ertelmezheto adat soraink legyenek. Ennek erdekeben probaltam a meglevo adatokat formazni. A pandas python csomag segitsegevel olyan muveleteket halytottam vegre amelyekel bizonyos szintu egyseges alakot ertem. Az ido rovidsege miatt kisebb nagyobb sikerrel a feladat adatisztitasi szakasaig erkeztem, ezeket a fajlokat a rendezett pontszamok mappaban taroltam.